In [1]:
import tensorflow as tf
import layers
import utils
import loss
import data_gen
import os

In [2]:
def yolo_model():
    input_layer = tf.keras.layers.Input((448, 448, 3))
    x = layers.SingleConv(64, 7, 2, True)(input_layer)
    x = layers.SingleConv(192, 3, 1, True)(x)
    x = layers.RepeatConv(256, 1)(x)
    x = layers.SingleConv(256, 1, 1, False)(x)
    x = layers.SingleConv(512, 3, 1, True)(x)
    x = layers.RepeatConv(512, 4)(x)
    x = layers.SingleConv(512, 1, 1, False)(x)
    x = layers.SingleConv(1024, 3, 1, True)(x)
    x = layers.RepeatConv(1024, 2)(x)
    x = layers.SingleConv(1024, 3, 1, False)(x)
    x = layers.SingleConv(1024, 3, 2, False)(x)
    x = layers.SingleConv(1024, 3, 1, False)(x)
    x = layers.SingleConv(1024, 3, 1, False)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(7 * 7 * 6, activation = 'relu')(x)
    output_layer = tf.keras.layers.Reshape((7, 7, 6))(x)
    
    return tf.keras.Model(input_layer, output_layer)

In [5]:
mdl = yolo_model()
mdl.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 448, 448, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_10 (SingleConv)     │ (None, 112, 112, 64)   │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_11 (SingleConv)     │ (None, 56, 56, 192)    │       111,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_conv_3 (RepeatConv)      │ (None, 56, 56, 256)    │       321,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_12 (SingleConv)     │ (None, 56, 56, 256)    │        66,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_13 (SingleConv)     │ (None, 28, 28, 512)    │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_conv_4 (RepeatConv)      │ (None, 28, 28, 512)    │     5,258,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_14 (SingleConv)     │ (None, 28, 28, 512)    │       264,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_15 (SingleConv)     │ (None, 14, 14, 1024)   │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_conv_5 (RepeatConv)      │ (None, 14, 14, 1024)   │    10,501,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_16 (SingleConv)     │ (None, 14, 14, 1024)   │     9,442,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_17 (SingleConv)     │ (None, 7, 7, 1024)     │     9,442,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_18 (SingleConv)     │ (None, 7, 7, 1024)     │     9,442,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_19 (SingleConv)     │ (None, 7, 7, 1024)     │     9,442,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 294)            │    14,752,038 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 7, 7, 6)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,960,742 (285.95 MB)

 Trainable params: 74,934,374 (285.85 MB)

 Non-trainable params: 26,368 (103.00 KB)

Make small model to test it works (above model crashes on local device):

In [3]:
def small_mdl_test():
    input_layer = tf.keras.layers.Input((448, 448, 3))
    x = layers.SingleConv(1, 7, 2, True)(input_layer)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(7 * 7 * 6, activation = 'relu')(x)
    output_layer = tf.keras.layers.Reshape((7, 7, 6))(x)
    return tf.keras.Model(input_layer, output_layer)
sml_mdl = small_mdl_test()

In [4]:
sml_mdl.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 448, 448, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ single_conv_1 (SingleConv)      │ (None, 112, 112, 1)    │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 294)            │     3,688,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 7, 7, 6)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,688,382 (14.07 MB)

 Trainable params: 3,688,380 (14.07 MB)

 Non-trainable params: 2 (8.00 B)

In [5]:
IM_PATH = r'squirrel_data\images'
LABEL_PATH = r'squirrel_data\labels'

In [6]:
im_list = os.listdir(IM_PATH)

In [7]:
data = data_gen.DataGen(im_list, True, 1, IM_PATH, LABEL_PATH)

In [8]:
sml_mdl.compile(optimizer = 'Adam', loss = loss.yolo_loss)

In [9]:
sml_mdl.fit(data, epochs = 5, batch_size = 1)

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 40.8146
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 354.4700
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 27.2142
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 12.5339
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 169.7112
